In [1]:
from spacy.tokens import DocBin
db=DocBin()
db.from_disk(path='res.spacy')

ValueError: [E1014] Error loading DocBin data. It doesn't look like the data is in DocBin (.spacy) format. If your data is in spaCy v2's JSON training format, convert it using `python -m spacy convert file.json .`.

In [3]:
import pandas as pd
df=pd.read_csv('result.csv')

In [4]:
n=(len(df)+1)//48
for i in range(48):
    s_data=df.iloc[i*n+1:(i+1)*n+1]
    s_data.to_csv(f'dataset/res{i}.csv', index=False)

In [6]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()

df=spark.read.option("header", True).csv('dataset/res*.csv')

In [14]:
df.orderBy('customer_id','timestamp').show(1000)
df.mapInPandas

+--------------------+-------+----------+--------------------+---------+----------------------------------------+---------+
|                  id|speaker| timestamp|         customer_id| sales_id|                                    text|mass_mask|
+--------------------+-------+----------+--------------------+---------+----------------------------------------+---------+
|14706056\t1\t1628...|   null|      null|                null|     null|                                    null|     null|
|14706048\t1\t1628...|   null|      null|                null|     null|                                    null|     null|
|14706064\t1\t1628...|   null|      null|                null|     null|                                    null|     null|
|16147401\t1\t1631...|  False|      null|                null|     null|                                    null|     null|
|14706001\t1\t1628...|   null|      null|                null|     null|                                    null|     null|
|1470603

In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='-1'
import tensorflow as tf
seq_len=256
m=128
record_spec = {
            "input_ids": tf.io.FixedLenFeature([seq_len], tf.int64),
            "labels": tf.io.FixedLenFeature([m], tf.int64),
            "token_type_ids": tf.io.FixedLenFeature([seq_len], tf.int64),
            "target_mapping": tf.io.FixedLenFeature([m,seq_len], tf.float32),
            # "perm_mask": tf.io.FixedLenFeature([m,seq_len], tf.int64),
        }
def _parse_function(example_proto):
    return tf.io.parse_single_example(example_proto, record_spec)
dataset=tf.data.TFRecordDataset(['/datafile/kaixuan/nlg/s0125.tfrecords'])
parsed_dataset = dataset.map(_parse_function)
for i in parsed_dataset.take(5):
    print(i['target_mapping'])


tf.Tensor(
[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]], shape=(128, 256), dtype=float32)
tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]], shape=(128, 256), dtype=float32)
tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(128, 256), dtype=float32)
tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(128, 256), dtype=float32)
tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]], shape=(128, 256), dtype=floa